Esse arquivo **Grupo 44_Turma_4DTAT_Fase4.ipynb** tem como objetivo consolidar as informações analíticas e os indicadores relevantes da **Análise dos dados de preço do Petróleo Brent** após o ano de 2000. A discussão detalhada desses indicadores pode ser encontrada no documento Word **Grupo 44 - Turma 4DTAT - Fase 4.docx**.

# Metodologia
Este projeto foi desenvolvido seguindo a metodologia CRISP-DM (Cross Industry Standard Process for Data Mining), amplamente adotada em projetos de análise de dados. Essa abordagem é estruturada em seis etapas principais, que são:

* **Análise do Negócio:** Focada no entendimento aprofundado do contexto e dos objetivos do negócio.
* **Análise dos Dados**:  Voltada para a compreensão e exploração inicial dos dados, garantindo alinhamento com os objetivos do negócio.
* **Preparação dos Dados:** Etapa dedicada ao pré-processamento e transformação dos dados para garantir qualidade e adequação ao modelo.
* **Modelagem:** Envolve a aplicação de técnicas analíticas para extrair valor dos dados e gerar insights significativos.
* **Avaliação:** Examina o desempenho do modelo desenvolvido, verificando se atende aos objetivos propostos.
* **Implementação:** Última etapa, onde os principais resultados são disponibilizados e aplicados no ambiente de produção.

Para esse notebook, temos as etapas de Análise do Negócio, Preparação dos Dados, Análise dos Dados, Modelagem e Avaliação dos Modelos elaborados.

# Análise do Negócio

O petróleo Brent é extraído do Mar do Norte e comercializado na Bolsa de Londres, sendo uma das principais referências mundiais junto ao WTI (West Texas Intermediate). Ambos são tipos de petróleo leve e doce, valorizados por sua fácil refinaria. A principal diferença entre os dois está na localização de extração e no mercado que refletem: enquanto o Brent é referência na Europa e no Mar do Norte, o WTI é focado no mercado dos EUA.

Mais da metade do petróleo mundial é cotado em termos do Brent, devido à facilidade de extração em campos offshore. Ele é negociado no mercado à vista (com liquidação imediata) ou no mercado futuro, onde contratos baseados nas cotações WTI e Brent são comprados por investidores e produtores, visando lucro ou proteção contra a volatilidade de preços.

O preço do petróleo é influenciado por fatores como a cotação do dólar, políticas da OPEC, níveis de produção e estoque, a saúde da economia global e acordos internacionais.
<font color=yellow> (Colocar fontes no word)

# Análise dos dados
Os dados deste projeto foram obtidos no site do IPEA e contêm os preços diários do barril de petróleo Brent, negociados em dias úteis, sem incluir despesas de frete e seguro. A tabela foi importada com a biblioteca Pandas e exportada para um arquivo CSV, garantindo acessibilidade e evitando problemas de indisponibilidade da fonte. As variáveis relevantes foram analisadas e exploradas a seguir.

## Importando bibliotecas

In [3]:
import warnings
warnings.filterwarnings(action = 'ignore')

# Importando pacotes
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import plotly.express as px
from plotly.subplots import make_subplots
import plotly.graph_objects as go

## Importando dados do IPEA


In [4]:
df = pd.read_html('http://www.ipeadata.gov.br/ExibeSerie.aspx?module=m&serid=1650971490&oper=view', skiprows=1, thousands='.', decimal=',')[0]
df.head()

,0,1
0,18/11/2024,74.35
1,15/11/2024,73.45
2,14/11/2024,73.39
3,13/11/2024,72.86
4,12/11/2024,72.56


## Tratamento de dados

In [5]:
def renomearColunas(df):
    df.rename(columns={0: 'data', 1: 'preco'}, inplace=True)
    return df

In [6]:
def tipoData(df):
    df['data'] = df['data'].str.replace('/', '-')
    df['data'] = pd.to_datetime(df['data'], format='%d-%m-%Y')
    return df

In [7]:
def excluirDadosNulos(df):
    if 0 not in df.isna().sum().values:
        print(f'{df.isna().sum().values.sum()} dados nulos excluídos')
        df = df.dropna()
    else:
        print('Sem dados nulos')
    return df

In [8]:
# Excluindo dados nulos
df = excluirDadosNulos(df)
df.head()

Sem dados nulos


,0,1
0,18/11/2024,74.35
1,15/11/2024,73.45
2,14/11/2024,73.39
3,13/11/2024,72.86
4,12/11/2024,72.56


In [9]:
# Renomear as Colunas
df = renomearColunas(df)
df.head()

,data,preco
0,18/11/2024,74.35
1,15/11/2024,73.45
2,14/11/2024,73.39
3,13/11/2024,72.86
4,12/11/2024,72.56


In [10]:
# Alterar o tipo da coluna Data
df = tipoData(df)
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 11302 entries, 0 to 11301
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype         
---  ------  --------------  -----         
 0   data    11302 non-null  datetime64[ns]
 1   preco   11302 non-null  float64       
dtypes: datetime64[ns](1), float64(1)
memory usage: 176.7 KB


## EDA

In [11]:
df_eda = df.sort_values('data', ascending=False).set_index('data')
df_eda.head()

,preco
data,
2024-11-18,74.35
2024-11-15,73.45
2024-11-14,73.39
2024-11-13,72.86
2024-11-12,72.56


In [12]:
template = 'ggplot2'

**Estatística Descritiva**

In [13]:
df_eda.describe()

,preco
count,11302.000000
mean,53.330436
std,33.155847
min,9.100000
25%,20.630000
50%,48.935000
75%,76.792500
max,143.950000


**Distribuição dos Dados**

In [14]:
import plotly.express as px

fig = px.histogram(
    data_frame=df_eda,
    x='preco',
    nbins=20,
    template=template,
    color_discrete_sequence=['#1E90FF']  # Azul claro
)
fig.update_traces(
    texttemplate='%{y}',  # Exibe os valores do eixo y como rótulos
    textposition='outside'  # Posição dos rótulos fora das barras
)
fig.update_layout(
    title='Histograma - Preço Petróleo Brent',
    xaxis_title='Preço',
    yaxis_title='Frequência',
    width=1000,
    height=350,
    margin=dict(l=50, r=10, t=40, b=40)
)
fig.show()


In [15]:
fig = px.box(
    data_frame=df_eda,
    x = 'preco',
    template = template,
    color_discrete_sequence = ['#6A5ACD']
)
fig.update_layout(
    title = 'Boxplot - Preço do Petróleo Brent',
    xaxis_title = 'Preço (US$)',
    yaxis_title = 'Volume',
    width = 1000,
    height = 350,
    margin=dict(l=50, r=10, t=40, b=40),
    template = template
)
fig.show()

####Análise da Série

In [16]:
from statsmodels.tsa.seasonal import seasonal_decompose # Decompor a série temporal

In [17]:
resultados = seasonal_decompose(df_eda, period=5)

In [18]:
media_movel = resultados.trend.to_frame()
sazonalidade = resultados.seasonal.to_frame()
residuo = resultados.resid.to_frame()

fig = make_subplots(rows = 4, cols = 1)
fig.add_trace(go.Scatter(x=df_eda.index, y=df.preco, name='Série', marker=dict(color='#2F4F4F')), row=1, col=1)
fig.add_trace(go.Scatter(x=media_movel.index, y=media_movel.trend, name='Media móvel', marker=dict(color='#00008B')), row=2, col=1)
fig.add_trace(go.Scatter(x=sazonalidade.index, y=sazonalidade.seasonal, name='Sazonalidade', marker = dict(color='#B300B3')), row=3, col=1)
fig.add_trace(go.Scatter(x=residuo.index, y=residuo.resid, name='Resíduo', marker=dict(color='#FFD700')), row=4, col=1)
fig.update_layout(
    title={
        'text': 'Decomposição - Série de Preços do Petróleo Brent',
        'x': 0.5,
        'xanchor': 'center'
    },
    width=1000,
    height=500,
    template=template
)
fig.show()

####Análise da Sazonalidade

In [19]:
df_eda = df_eda.loc[df_eda.index >= '2023-01-01']
resultados_teste = seasonal_decompose(df_eda, period=5)
sazonalidade_teste = resultados_teste.seasonal.to_frame()

fig = px.line(
    sazonalidade_teste,
    template=template,
    color_discrete_sequence=['#B300B3']
)
fig.update_layout(
    title='Sazonalidade - Série de Preços do Petróleo Brent',
    xaxis_title='Período',
    yaxis_title='Sazonalidade',
    showlegend=False,
    width=1000,
    height=400,
    margin=dict(l=100, r=20, t=80, b=80)
)
fig.show()

**Análise das Oscilações**

O petróleo Brent sofreu oscilações significativas em três momentos recentes:

**2008:** Durante a crise financeira global, desencadeada pela especulação imobiliária nos EUA (subprime), houve uma queda drástica na demanda e no preço do petróleo. Após o colapso do Lehman Brothers, os investidores abandonaram commodities como o petróleo, buscando liquidez, enquanto o consumo global de combustíveis despencava.

**2014:** A superprodução, especialmente do petróleo de xisto dos EUA, combinada com a menor demanda na Europa e Ásia, derrubou os preços. A OPEP recusou reduzir a produção para preservar sua participação de mercado, prejudicando economias dependentes do petróleo, como Venezuela, Rússia e Irã.

**2020:** A pandemia da COVID-19 provocou uma redução global no consumo de petróleo devido ao isolamento social, resultando em queda de preços. Divergências entre a Arábia Saudita e a Rússia sobre cortes na produção levaram a uma guerra de preços, com a Arábia Saudita aumentando a oferta e reduzindo drasticamente os valores.

Esses eventos mostram como fatores econômicos, geopolíticos e crises globais afetam diretamente o mercado do petróleo Brent.

<font color=yellow> (Colocar fontes no word)

# Preparação dos dados

In [20]:
!pip install statsforecast

In [21]:
# Importando os pacotes Estatísticos
from statsforecast import StatsForecast
from statsforecast.models import AutoARIMA
from prophet import Prophet

**Tratamento do DataFrame para aplicação do ARIMA**

A biblioteca StatsForecast oferece uma interface simples para previsão de séries temporais, suportando os modelos estatísticos mais populares.

In [22]:
df_tratado = df[['data', 'preco']].rename(columns={'data': 'ds', 'preco': 'y'})
df_tratado['unique_id'] = 'Preco'
df_tratado.dropna(inplace=True)
df_tratado.head()

,ds,y,unique_id
0,2024-11-18,74.35,Preco
1,2024-11-15,73.45,Preco
2,2024-11-14,73.39,Preco
3,2024-11-13,72.86,Preco
4,2024-11-12,72.56,Preco


Separação da base de treino e teste para aplicação estatística

In [23]:
treino_arima = df_tratado.loc[(df_tratado['ds'] >= '2000-01-01') & (df_tratado['ds'] < '2024-11-12')] # Dados para treino
teste_arima = df_tratado.loc[df_tratado['ds'] >= '2024-11-12'] # Dados para validação, nesse caso, 5 dias. Quanto maior o período de predição, maior a largura de banda (provável erro)
h = teste_arima['ds'].nunique() # Quantidade de dias para prever (treino 5 dias)
h

5

In [24]:
teste_arima.shape, treino_arima.shape

((5, 3), (8097, 3))

# Modelagem

### ARIMA

O modelo ARIMA (AutoRegressive Integrated Moving Average) no StatsForecast é utilizado para previsão de séries temporais. Ele combina três componentes principais:

1. AR (AutoRegressive): Modela a dependência linear entre observações passadas.

2. I (Integrated): Torna a série estacionária ao calcular diferenças entre observações.

3. MA (Moving Average): Modela o erro como uma combinação linear de erros passados.

A implementação no StatsForecast automatiza o processo de seleção dos parâmetros ideais (p, d, q) e permite prever múltiplas séries ao mesmo tempo, sendo eficiente para lidar com grandes volumes de dados.

In [25]:
# Aplicação do Treino e Teste para Modelo Arima
modelo_arima = StatsForecast(models=[AutoARIMA(season_length=5)], freq='B', n_jobs=-1)
modelo_arima.fit(treino_arima)
previsao_arima = modelo_arima.predict(h=h, level=[90])
previsao_arima.ds = teste_arima.ds.to_list()
previsao_arima = previsao_arima.reset_index().merge(teste_arima, on =['ds', 'unique_id'], how='left')
previsao_arima

,unique_id,ds,AutoARIMA,AutoARIMA-lo-90,AutoARIMA-hi-90,y
0,Preco,2024-11-18,72.841614,70.161186,75.522041,74.35
1,Preco,2024-11-15,73.063278,69.587082,76.539474,73.45
2,Preco,2024-11-14,72.819267,68.897949,76.740578,73.39
3,Preco,2024-11-13,72.576576,68.248222,76.904922,72.86
4,Preco,2024-11-12,72.686501,67.833626,77.539375,72.56


Legenda para as colunas acima:

**Datas previstas (ds):** Correspondem ao período de validação (5 dias, de teste_arima).

**Valores previstos (AutoARIMA):** As previsões de preços feitas pelo modelo ARIMA.

**Intervalo de Confiança (AutoARIMA-lo-90 e AutoARIMA-hi-90):**
Limites inferior e superior do intervalo de 90% de confiança para cada previsão.

**Valores reais (y):** Coluna adicionada após o merge, mostrando os valores reais do período de teste (caso existam).

**unique_id (caso presente):** Identificador das séries temporais, importante em projetos com múltiplas séries.

In [26]:
# Linha contínua elaborada para banda de confiança
banda_superior_arima = previsao_arima['AutoARIMA-hi-90'].tolist() # Banda limite superior
banda_inferior_arima = previsao_arima['AutoARIMA-lo-90'].tolist() # Banda limite inferior
banda_arima_total = banda_superior_arima + banda_inferior_arima[::-1] # Concatenar banda superior e inferior invertida
indice_banda_arima = previsao_arima.ds.tolist()
indice_banda_arima = indice_banda_arima + indice_banda_arima[::-1] # Concatenar datas em ordem crescente com a decrescente

In [27]:
# Concatenar variáveis de Treino e Teste
dados_treino_px = treino_arima.drop(columns = 'unique_id').set_index('ds')
dados_treino_px = dados_treino_px[(dados_treino_px.index >= '2024-08-01') & (dados_treino_px.index < '2024-11-18')]
dados_teste_px = teste_arima.drop(columns = 'unique_id').set_index('ds')
base_total_arima_px = pd.concat([dados_teste_px, dados_treino_px]).rename(columns = {'y': 'Preço'})
base_total_arima_px.head()

,Preço
ds,
2024-11-18,74.35
2024-11-15,73.45
2024-11-14,73.39
2024-11-13,72.86
2024-11-12,72.56


In [28]:
fig = px.line(
    base_total_arima_px,  # Preço dos valores utilizados para Treino e Teste (tudo)
    template=template,
    color_discrete_sequence=['#00008B']
)
fig.add_scatter(
    x=indice_banda_arima,
    y=banda_arima_total,  # Informação das Bandas
    name='Banda de confiança',
    mode='lines',
    line_color='#B22222',
    line_width=0,
    opacity=0.5,
    fill='toself'
)
fig.add_scatter(
    x=previsao_arima.ds, # Informação do valor de Teste (Previsão)
    y=previsao_arima['AutoARIMA'],
    name='Preço previsto',
    marker=dict(color='#B22222')
)
fig.update_layout(
    title='ARIMA',
    xaxis_title='Período',
    yaxis_title='Preço do Petróleo Brent',
    legend_title='',
    legend=dict(
        orientation="h",  # Orientação horizontal
        yanchor="top",    # Âncora no topo da legenda
        y=-0.6,           # Posição vertical abaixo do gráfico
        xanchor="center", # Âncora no centro horizontal
        x=0.5             # Posição horizontal centralizada
    ),
    width=1000,
    height=300,
    margin=dict(l=100, r=20, t=80, b=80)
)
fig.show()

# Avaliação do Modelo

## Função WMAPE
O WMAPE (Weighted Mean Absolute Percentage Error) é uma métrica de avaliação para modelos de previsão que calcula o erro absoluto médio ponderado pelos valores reais. Ele é mais robusto que o MAPE, pois evita distorções quando os valores reais são pequenos ou próximos de zero. Quanto menor o WMAPE, melhor o desempenho do modelo. É amplamente usado por ser intuitivo e apresentar resultados em porcentagem, facilitando a interpretação.

In [29]:
# Validação do ARIMA
def wmape(y_true, y_pred):
    return np.abs(y_true - y_pred).sum() / np.abs(y_true).sum()

In [30]:
wmape_arima = wmape(previsao_arima['y'].values, previsao_arima['AutoARIMA'].values)
print(f'WAPE: {wmape_arima}')

WAPE: 0.007844207947638059


O WMAPE mede a proporção do erro em relação aos valores reais, sendo que valores mais próximos de zero indicam maior precisão do modelo. Com um WMAPE de 0.00657, o desempenho do modelo pode ser considerado bom.

# Exportando o Modelo



In [ ]:
import joblib
from statsforecast import StatsForecast
from statsforecast.models import AutoARIMA

df_tratado = df[['data', 'preco']].rename(columns={'data': 'ds', 'preco': 'y'})
df_tratado['unique_id'] = 'Preco'
df_tratado.dropna(inplace=True)

treino = df_tratado.loc[df_tratado['ds'] >= '2000-01-01'] # Novos dados treino
h = 5

modelo_arima_brent = StatsForecast(models=[AutoARIMA(season_length=5)], freq='B', n_jobs=-1)
modelo_arima_brent.fit(treino)
joblib.dump(modelo_arima_brent, 'modelos/auto_arima_brent.joblib') # Exportação para pasta modelos
#['auto_arima_brent.joblib'] 

['modelos/auto_arima_brent.joblib']

In [ ]:
# Visualizando os dados de previsão do Modelo
modelo = joblib.load('modelos/auto_arima_brent.joblib')
previsao_arima = modelo.predict(h=h, level=[90])
previsao_arima = previsao_arima[['ds', 'AutoARIMA']].reset_index(drop=True).rename(columns={'ds': 'data', 'AutoARIMA': 'preco_previsto_brent'})
previsao_arima['preco_previsto_brent'] = [int(n * 100) / 100 for n in previsao_arima['preco_previsto_brent']]
previsao_arima

,data,preco_previsto_brent
0,2024-11-19,74.18
1,2024-11-20,74.10
2,2024-11-21,74.12
3,2024-11-22,74.23
4,2024-11-25,74.21
